# Source Location in 3D

In [1]:
import hmc_tomography
import matplotlib.pyplot as plt
import numpy
from hmc_tomography.Distributions import SourceLocation

In [2]:
import psvWave


settings = psvWave.get_dictionary()

# Create a 10x10 km domain
settings["domain"]["nx_inner"] = 500
settings["domain"]["nz_inner"] = 500
settings["domain"]["dx"] = 20
settings["domain"]["dz"] = 20
settings["domain"]["dt"] = 0.001
settings["domain"]["nt"] = 10000

settings["sources"]["peak_frequency"] = 10
settings["sources"]["n_shots"] = 3
settings["sources"]["n_sources"] = 3
settings["sources"]["moment_angles"] = [48, 79, 12]
settings["sources"]["ix_sources"] = [0, 200, 350]  # in grid index
settings["sources"]["iz_sources"] = [200, 250, 375]  # in grid index
settings["sources"]["which_source_to_fire_in_which_shot"] = [[0], [1], [2]]

settings["medium"]["scalar_vs"] = 0
settings["receivers"]["ix_receivers"] = [65, 100, 300, 420]
settings["receivers"]["iz_receivers"] = [0, 0, 0, 0]

settings["inversion"]["snapshot_interval"] = 1000

settings

ModuleNotFoundError: No module named '__psvWave_cpp'

In [ ]:
receivers_x = (
    numpy.array(settings["receivers"]["ix_receivers"])
    * numpy.array(settings["domain"]["dx"])
    / 1000.0
)[None, :]
receivers_z = (
    numpy.array(settings["receivers"]["iz_receivers"])
    * numpy.array(settings["domain"]["dz"])
    / 1000.0
)[None, :]

sources_x = (
    numpy.array(settings["sources"]["ix_sources"])
    * numpy.array(settings["domain"]["dx"])
    / 1000.0
)[None, :]
sources_z = (
    numpy.array(settings["sources"]["iz_sources"])
    * numpy.array(settings["domain"]["dz"])
    / 1000.0
)[None, :]


numpy.save("bin/5/receivers_x.npy", receivers_x)
numpy.save("bin/5/receivers_z.npy", receivers_z)
numpy.save("bin/5/sources_x.npy", sources_x)
numpy.save("bin/5/sources_z.npy", sources_z)

In [ ]:
psvWave.write_dictionary("bin/5/fd_settings.ini", settings)
fdModel = psvWave.fdModel("bin/5/fd_settings.ini")

fdModel.plot_domain()

plt.gca().invert_yaxis()

In [ ]:
from scipy.ndimage import gaussian_filter, gaussian_filter1d

vp, vs, rho = fdModel.get_parameter_fields()

vp = 2000 * numpy.ones_like(vp)
rho = 1500 * numpy.ones_like(rho)

numpy.random.seed(321321)
anomalies = gaussian_filter(numpy.random.randn(*vp.shape), sigma=4)
anomalies = 200 * anomalies / numpy.abs(anomalies).max()
vp = vp + anomalies

anomalies_2 = gaussian_filter(numpy.random.randn(*vp.shape), sigma=4)
anomalies_2 = 200 * anomalies_2 / numpy.abs(anomalies_2).max()
rho = rho - 0.5 * anomalies + 0.5 * anomalies_2

fdModel.set_parameter_fields(vp, vs, rho)

numpy.save("bin/5/vp.npy", vp)
numpy.save("bin/5/vs.npy", vs)
numpy.save("bin/5/rho.npy", rho)

axes = fdModel.plot_fields(
    vmax=[vp.mean() * 1.2, vs.mean() * 1.2, rho.mean() * 1.2],
    vmin=[vp.mean() * 0.8, vs.mean() * 0.8, rho.mean() * 0.8],
)
for axis in axes:
    axis.invert_yaxis()

In [ ]:
# Create 'true' data
# print("Faking observed data")
for i_shot in range(fdModel.n_shots):
    fdModel.forward_simulate(i_shot, omp_threads_override=6)

# Cheating of course, as this is synthetically generated data.
ux_obs, uz_obs = fdModel.get_synthetic_data()

numpy.save("bin/5/ux_obs.npy", ux_obs)
numpy.save("bin/5/uz_obs.npy", uz_obs)

_ = fdModel.plot_data(data=(ux_obs, uz_obs), exagerration=1)

In [ ]:
ux_obs = numpy.load("bin/5/ux_obs.npy")
uz_obs = numpy.load("bin/5/uz_obs.npy")

numpy.random.seed(32312)

ux_obs_noise_addded = ux_obs + 3e-4 * gaussian_filter1d(
    numpy.random.randn(*ux_obs.shape), sigma=2, axis=2
)
uz_obs_noise_addded = uz_obs + 3e-4 * gaussian_filter1d(
    numpy.random.randn(*ux_obs.shape), sigma=2, axis=2
)

_ = fdModel.plot_data(data=(ux_obs_noise_addded, uz_obs_noise_addded), exagerration=1)

In [ ]:
time_step = 3
plt.figure()

particle_velocity = [None, None, None]
particle_velocity_max = [None, None, None]
for i_shot in range(fdModel.n_shots):
    particle_velocity[i_shot] = (
        fdModel.get_snapshots()[0][i_shot, time_step, :, :].T ** 2
        + fdModel.get_snapshots()[1][i_shot, time_step, :, :].T ** 2
    )
    particle_velocity_max[i_shot] = numpy.max(numpy.abs(particle_velocity[i_shot]))

field_extr = numpy.max(particle_velocity_max)

for i_shot in range(fdModel.n_shots):

    field = particle_velocity[i_shot]

    fdModel.plot_domain(axis=plt.gca())

    extent = fdModel.get_extent(True)
    plt.imshow(
        field,
        vmin=-field_extr,
        vmax=field_extr,
        cmap=plt.get_cmap("seismic"),
        extent=[
            extent[0],
            extent[1],
            extent[3],
            extent[2],
        ],
    )

    cbar = plt.colorbar()
    cbar.set_label("particle velocity", rotation=90)

    plt.gca().invert_yaxis()
    plt.title(
        f"Shot {i_shot+1}, time {time_step * settings['inversion']['snapshot_interval'] * settings['domain']['dt']} seconds"
    )
    plt.show()

In [ ]:
velocity = ux_obs_noise_addded ** 2 + uz_obs_noise_addded ** 2

exceed_noise = numpy.empty_like(velocity)

for i_shot in range(fdModel.n_shots):
    plt.figure(figsize=(16, 8))
    noise_level = velocity[i_shot, :, 0:150].max()
    for i_rec in range(ux_obs_noise_addded.shape[1]):
        plt.plot(
            numpy.linspace(0, fdModel.dt * fdModel.nt, fdModel.nt),
            velocity[i_shot, i_rec, :],
        )
    plt.ylim([0, noise_level * 5])
    plt.xlim([2, 5.5])

    exceed_noise[i_shot, :, :] = velocity[i_shot, :, :] > 5 * noise_level

In [ ]:
for i_shot in range(fdModel.n_shots):
    plt.figure()
    plt.plot(exceed_noise[i_shot, :, :].T)

In [ ]:
arrival_times = numpy.empty((fdModel.n_shots, ux_obs_noise_addded.shape[1]))

for i_shot in range(fdModel.n_shots):
    for i_rec in range(ux_obs_noise_addded.shape[1]):
        arrival_times[i_shot, i_rec] = (
            fdModel.dt * numpy.where(exceed_noise[i_shot, i_rec, :])[0][0] + 0.05
        )

numpy.save("bin/5/arrival_times.npy", arrival_times)

In [ ]:
arrival_times = numpy.load("bin/5/arrival_times.npy")

receivers_x = numpy.load("bin/5/receivers_x.npy")
receivers_z = numpy.load("bin/5/receivers_z.npy")
sources_x = numpy.load("bin/5/sources_x.npy")
sources_z = numpy.load("bin/5/sources_z.npy")

arrival_times

In [ ]:
likelihood = SourceLocation(
    receivers_x, receivers_z, arrival_times, numpy.ones_like(arrival_times) * 0.1
)
likelihood.describe()

In [ ]:
likelihood.plot_data()
plt.legend()

In [ ]:
from hmc_tomography.Distributions import (
    Normal,
    Uniform,
    CompositeDistribution,
    BayesRule,
)

event_1_x_prior = Normal(0, 5)
event_2_x_prior = Normal(4, 5)
event_3_x_prior = Normal(6, 5)

max_speed = 5  # km/s

# This is the time at any station, some wave is observed, meaning that origin time has to be before this.
latest_first_arrival = 4  # sec

event_1_z_prior = Uniform([0], [latest_first_arrival * max_speed])
event_2_z_prior = event_1_z_prior
event_3_z_prior = event_1_z_prior
event_1_t_prior = Uniform([-1], [latest_first_arrival])
event_2_t_prior = event_1_t_prior
event_3_t_prior = event_1_t_prior

speed_prior = Normal(2.0, 1.0 ** 2, lower_bounds=[0.5], upper_bounds=[max_speed])

prior = CompositeDistribution(
    [
        event_1_x_prior,
        event_1_z_prior,
        event_1_t_prior,
        event_2_x_prior,
        event_2_z_prior,
        event_2_t_prior,
        event_3_x_prior,
        event_3_z_prior,
        event_3_t_prior,
        speed_prior,
    ]
)

posterior = BayesRule([prior, likelihood])

In [ ]:
from hmc_tomography.Samplers import HMC, RWMH

rwmh = RWMH()

rwmh.sample(
    "bin/5/rwmh_trial.h5",
    posterior,
    initial_model=numpy.array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2])[:, None],
    autotuning=True,
    proposals=100000,
    online_thinning=1,
    overwrite_existing_file=True,
    max_time=3,
)

hmc = HMC()

hmc.sample(
    "bin/5/hmc_trial.h5",
    posterior,
    initial_model=numpy.array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2])[:, None],
    autotuning=True,
    proposals=100000,
    online_thinning=1,
    overwrite_existing_file=True,
    max_time=3,
)

In [ ]:
with hmc_tomography.Samples("bin/5/rwmh_trial.h5", burn_in=0) as samples_no_burn:
    line_rwmh = plt.plot(
        numpy.linspace(0, 1, samples_no_burn[0, :].size),
        samples_no_burn[7:-1, :].T,
        "r",
    )

with hmc_tomography.Samples("bin/5/hmc_trial.h5", burn_in=0) as samples_no_burn:
    line_hmc = plt.plot(
        numpy.linspace(0, 1, samples_no_burn[0, :].size),
        samples_no_burn[7:-1, :].T,
        "k",
    )

plt.xlabel("fraction of short chain")

In [ ]:
hmc = HMC()

hmc.sample(
    "bin/5/hmc_5_minutes.h5",
    posterior,
    initial_model=numpy.array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2])[:, None],
    autotuning=True,
    proposals=200000,
    online_thinning=10,
    overwrite_existing_file=True,
    max_time=10,
)

In [ ]:
with hmc_tomography.Samples("bin/5/hmc_5_minutes.h5", burn_in=0) as samples:
    plt.plot(samples[-1, :100])

In [ ]:
with hmc_tomography.Samples("bin/5/hmc_5_minutes.h5", burn_in=20) as samples:
    samples_n = samples.numpy

hmc_tomography.Visualization.marginal_grid(samples_n, [0, 1, 2, 9])
hmc_tomography.Visualization.marginal_grid(samples_n, [3, 4, 5, 9])
hmc_tomography.Visualization.marginal_grid(samples_n, [6, 7, 8, 9])

In [ ]:
data_samples = 50

print(f"We have {samples_n.shape[1]} samples, let's select {data_samples}.")

selection = numpy.random.randint(0, samples_n.shape[1], data_samples)

# We also discard the last dimension, misfit
selection = samples_n[:-1, selection]

from cycler import cycler

figure = plt.figure(figsize=(8, 8))
likelihood.plot_data(figure=figure)

cmap = plt.get_cmap("tab10")

for i in range(data_samples):

    data = likelihood.forward_vector(selection[:, i, None])

    for event in range(3):
        plt.scatter(
            likelihood.receiver_array_x.T,
            data[event, :],
            color=cmap(event),
            alpha=1.0 / 100,
        )

_ = plt.ylim([2, 5.2])

In [ ]:
from matplotlib.colors import LinearSegmentedColormap

plt.figure(figsize=(10, 10))
for event in range(3):
    ncolors = 256
    color_array = plt.get_cmap("seismic")(range(ncolors))
    color_array[:, -1] = numpy.linspace(0.0, 0.5, ncolors)
    color_array[:, :-1] = cmap(event)[:-1]
    map_object = LinearSegmentedColormap.from_list(
        name="event_specific", colors=color_array
    )
    plt.register_cmap(cmap=map_object)

    plt.hist2d(
        samples_n[event * 3],
        samples_n[event * 3 + 1],
        range=[[-5, 15], [0, 10]],
        bins=50,
        cmap="event_specific",
    )

plt.scatter(
    likelihood.receiver_array_x, likelihood.receiver_array_z, c="r", label="receivers"
)

plt.gca().invert_yaxis()

for event in range(3):
    plt.scatter(
        sources_x[0, event],
        sources_z[0, event],
        color=cmap(event),
        s=100,
        label=f"Event {event} true location",
    )

plt.legend()
# plt.xlim([likelihood.receiver_array_x.min(), likelihood.receiver_array_x.max()])

In [ ]:
threshold_samples = samples_n[:, samples_n[-2, :] < 2.1]

plt.figure(figsize=(10, 10))
for event in range(3):
    ncolors = 256
    color_array = plt.get_cmap("seismic")(range(ncolors))
    color_array[:, -1] = numpy.linspace(0.0, 0.5, ncolors)
    color_array[:, :-1] = cmap(event)[:-1]
    map_object = LinearSegmentedColormap.from_list(
        name="event_specific", colors=color_array
    )
    plt.register_cmap(cmap=map_object)

    plt.hist2d(
        threshold_samples[event * 3],
        threshold_samples[event * 3 + 1],
        range=[[-5, 15], [0, 10]],
        bins=50,
        cmap="event_specific",
    )

plt.scatter(
    likelihood.receiver_array_x, likelihood.receiver_array_z, c="r", label="receivers"
)

plt.gca().invert_yaxis()

for event in range(3):
    plt.scatter(
        sources_x[0, event],
        sources_z[0, event],
        color=cmap(event),
        s=100,
        label=f"Event {event} true location",
    )

plt.legend()
# plt.xlim([likelihood.receiver_array_x.min(), likelihood.receiver_array_x.max()])

In [ ]:
chains = 6

# 3 samplers
samplers = [HMC() for i in range(chains)]

# 3 posteriors (that are not the same!)
posteriors = [posterior] * chains

# 3 separate sample files
filenames = [f"bin/5/hmc_parallel_{i}.h5" for i in range(chains)]


obj = (
    hmc_tomography.Samplers.ParallelSampleSMP()
    .sample(
        samplers,
        filenames,
        posteriors,
        overwrite_existing_files=True,
        proposals=20000,
        exchange=True,
        initial_model=numpy.array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2])[:, None],
        kwargs={"autotuning": True, "max_time": 10, "online_thinning": 10},
    )
    .print_results()
)

In [ ]:
samples_n = [None] * len(filenames)

for i_chain, filename in enumerate(filenames):
    with hmc_tomography.Samples(filename, burn_in=0) as samples:
        plt.plot(samples[-1, 20:100], alpha=0.5)

        samples_n[i_chain] = samples[:-1, 20:]

samples_combined = numpy.hstack(samples_n)

In [ ]:
from matplotlib.colors import LinearSegmentedColormap

plt.figure(figsize=(10, 10))
for event in range(3):
    ncolors = 256
    color_array = plt.get_cmap("seismic")(range(ncolors))
    color_array[:, -1] = numpy.linspace(0.0, 0.5, ncolors)
    color_array[:, :-1] = cmap(event)[:-1]
    map_object = LinearSegmentedColormap.from_list(
        name="event_specific", colors=color_array
    )
    plt.register_cmap(cmap=map_object)

    plt.hist2d(
        samples_combined[event * 3],
        samples_combined[event * 3 + 1],
        range=[[-5, 15], [0, 10]],
        bins=50,
        cmap="event_specific",
    )

plt.scatter(
    likelihood.receiver_array_x, likelihood.receiver_array_z, c="r", label="receivers"
)

plt.gca().invert_yaxis()

for event in range(3):
    plt.scatter(
        sources_x[0, event],
        sources_z[0, event],
        color=cmap(event),
        s=100,
        label=f"Event {event} true location",
    )

plt.legend()
# plt.xlim([likelihood.receiver_array_x.min(), likelihood.receiver_array_x.max()])

In [ ]:
threshold_samples = samples_combined[:, samples_combined[-2, :] < 2.1]

plt.figure(figsize=(10, 10))
for event in range(3):
    ncolors = 256
    color_array = plt.get_cmap("seismic")(range(ncolors))
    color_array[:, -1] = numpy.linspace(0.0, 0.5, ncolors)
    color_array[:, :-1] = cmap(event)[:-1]
    map_object = LinearSegmentedColormap.from_list(
        name="event_specific", colors=color_array
    )
    plt.register_cmap(cmap=map_object)

    plt.hist2d(
        threshold_samples[event * 3],
        threshold_samples[event * 3 + 1],
        range=[[-5, 15], [0, 10]],
        bins=50,
        cmap="event_specific",
    )

plt.scatter(
    likelihood.receiver_array_x, likelihood.receiver_array_z, c="r", label="receivers"
)

plt.gca().invert_yaxis()

for event in range(3):
    plt.scatter(
        sources_x[0, event],
        sources_z[0, event],
        color=cmap(event),
        s=100,
        label=f"Event {event} true location",
    )

plt.legend()
# plt.xlim([likelihood.receiver_array_x.min(), likelihood.receiver_array_x.max()])